In [19]:
import pandas as pd
import os
from statsmodels.tsa.arima_model import ARIMA
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import json

In [20]:
from numpy.random import seed
seed(1)

In [21]:
trips_data = pd.read_csv(os.path.join('data','trips_time_series_all_inter.csv'),index_col=0)
trips_data= trips_data.fillna(0)

In [22]:
trips_data.head()

,20200221_00,20200221_01,20200221_02,20200221_03,20200221_04,20200221_05,20200221_06,20200221_07,20200221_08,20200221_09,...,20201130_14,20201130_15,20201130_16,20201130_17,20201130_18,20201130_19,20201130_20,20201130_21,20201130_22,20201130_23
01001_AM,291.654,129.650,123.822,280.810,282.418,518.179,736.074,1253.000,981.544,893.510,...,1307.238,1026.083,1073.847,1147.206,650.740,648.024,563.314,735.201,258.461,230.591
01002,192.529,149.992,50.547,73.097,208.114,616.786,732.075,779.587,638.112,734.767,...,701.593,598.760,615.822,826.882,580.843,378.512,381.695,407.566,124.729,106.744
01010_AM,92.726,50.822,37.895,19.586,91.560,396.531,457.180,591.326,516.919,471.008,...,495.460,610.121,559.796,476.880,354.807,215.711,216.593,277.574,119.830,52.331
01031_AM,97.674,64.901,92.150,62.669,109.056,208.462,469.924,691.565,645.793,789.906,...,1085.613,628.526,682.578,582.456,603.958,344.157,286.541,269.655,159.443,118.029
01036,173.565,118.227,204.784,219.318,409.485,779.103,1091.613,969.506,845.084,875.701,...,1070.542,815.612,681.116,746.917,704.784,413.711,441.988,575.437,319.760,68.968


In [23]:
def forecast_ARIMA_fn(X):
    size = 100 #int(len(X) * 0.9)
    time_horizon= size + 1
    train, test = X[0:size], X[size:time_horizon]
    history = [x for x in train]
    predictions = list()
    # walk-forward validation
    for t in tqdm_notebook(range(len(test)), leave=False):
        model = ARIMA(history, order=(12,1,1))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
        #print('predicted=%f, expected=%f' % (yhat, obs))
    # evaluate forecasts
    mae_ = mean_absolute_error(test, predictions)
    mse_ = mean_squared_error(test, predictions)
    rmse_ = mean_squared_error(test, predictions, squared = False)
    #print('Test MAE: %.3f' % mse)
    return mae_, mse_, rmse_

In [24]:
metrics_arima = []
for i in tqdm_notebook(range(0,500)):
    try:
        X = trips_data.iloc[i].values
        mae_, mse_, rmse_ = forecast_ARIMA_fn(X)
        #print(trips_data.iloc[i].name, mae_, mse_, rmse_)
        metrics_arima.append((trips_data.iloc[i].name, mae_, mse_, rmse_))
    except:
        print("Error in interation {}".format(i))

/home/fterroso/python/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


/home/fterroso/python/lib/python3.6/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:695: RuntimeWarning: divide by zero encountered in log
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:410: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:496: RuntimeWarning: invalid value encountered in double_scalars
  dx = x[i] - x0[i]  # Recompute dx as exactly representable number.


Error in interation 12


/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:695: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:410: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:496: RuntimeWarning: invalid value encountered in double_scalars
  dx = x[i] - x0[i]  # Recompute dx as exactly representable number.


...

/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:695: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:410: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:496: RuntimeWarning: invalid value encountered in double_scalars
  dx = x[i] - x0[i]  # Recompute dx as exactly representable number.


Error in interation 484


/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:695: RuntimeWarning: divide by zero encountered in true_divide
  invmacoefs = -np.log((1-macoefs)/(1+macoefs))
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:668: RuntimeWarning: invalid value encountered in true_divide
  newparams = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/statsmodels/tsa/tsatools.py:669: RuntimeWarning: invalid value encountered in true_divide
  tmp = ((1-np.exp(-params))/(1+np.exp(-params))).copy()
/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:410: RuntimeWarning: invalid value encountered in subtract
  dx = ((x0 + h) - x0)


Error in interation 494


/home/fterroso/python/lib/python3.6/site-packages/scipy/optimize/_numdiff.py:496: RuntimeWarning: invalid value encountered in double_scalars
  dx = x[i] - x0[i]  # Recompute dx as exactly representable number.


In [25]:
arima_test_metrics_df = pd.DataFrame.from_records(metrics_arima, columns='area_id arima-MAE arima-MSE arima-RMSE'.split())

In [26]:
arima_test_metrics_df.head()

,area_id,arima-MAE,arima-MSE,arima-RMSE
0,01001_AM,81.324498,6613.673944,81.324498
1,01002,130.484099,17026.100077,130.484099
2,01010_AM,43.395710,1883.187646,43.395710
3,01031_AM,21.747880,472.970297,21.747880
4,01036,321.574864,103410.393304,321.574864


In [27]:
arima_test_metrics_df.mean()

arima-MAE       162.771901
arima-MSE     81142.684398
arima-RMSE      162.771901
dtype: float64

In [28]:
arima_test_metrics_df.to_csv(os.path.join('/','data', 'arima_test_metrics_0_500_2.csv'))

In [29]:
print("That's all folks")

That's all folks
